<a href="https://colab.research.google.com/github/ferchomuri/archi/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>